In [4]:
import torch
from torch import nn
import numpy as np
from typing import Any, Optional, Tuple, Type

In [34]:
# 用高斯随机生成随机位置编码

# 关于_position_embedding_encoding，它的输入都是实际坐标，因此在forward和forward函数调用之前要对coords进行处理，形成[d1, d2, ..., 2]的shape
# 简单来说就是shape=[batch_size, N, 2]形状的坐标coords

class PositionEmbeddingRandom(nn.Module):

    def __init__(self,
                 num_pos_feats: int = 64, # 位置编码的特征维度
                 scale: Optional[float] = None
                 ) -> None:
        super().__init__()
        if scale is None or scale <= 0.0:
            scale = 1.0
        self.register_buffer( # 用到了register_buffer，即它不是一个参数，而是一个在前向传播过程中需要的固定值
            "positional_encoding_gaussian_matrix", # 注册了一个名为这个的缓冲区张量
            scale * torch.randn((2, num_pos_feats)), # 大小为(2, num_pos_feats=64)的高斯矩阵，并用scale缩放
        )

    # 对输入的坐标做随机位置编码，要求输入坐标已经标准化在[0,1]之间。该方法要求传入的是实际坐标coords，而不是坐标的size，即coords_size
    def _position_embedding_encoding(self,
                     coords: torch.Tensor # shape = [d1, d2, ..., 2]，前面的dx是网格，最后一个2存储的是点坐标，例如[0.1, 0.3]，前面的所有维度是用来储存点坐标的列表，例如[batch_size, N, 2] = [3, 3, 2]
                     ) -> torch.Tensor:

        coords = 2 * coords - 1 # 将coords的范围从[0,1]放缩到[-1,1]
        coords = coords @ self.positional_encoding_gaussian_matrix # 例如[3, 3, 2] * [2, 64] = [3, 3, 64]
        coords = 2 * np.pi * coords # shape = [3, 3, 64]不变
        
        return torch.cat([torch.sin(coords), torch.cos(coords)], dim=-1) # 放进sin和cos之后沿着最后一个维度拼成一个新的张量，shape = [3, 3, 128]

    # 生成指定大小的网格位置编码，以_position_embedding_encoding()的例子为例，网格就应该是[3, 3]，输入参数是一个包含batch_size和N的元组Tuple(batch_size, N),以下简化为h和w的元组Tuple(h, w)
    # 传入的是坐标size，即coords_size
    def forward(self, 
                coords_size: Tuple[int, int] # 这个coords_size其实是(batch_size, N)，N为每个样本中的点数
                ) -> torch.Tensor:

        h, w = coords_size # 根据输入读取h和w
        device: Any = self.positional_encoding_gaussian_matrix.device
        
        #创建一个shape=[h, w]的全1张量，并储存在device上
        grid = torch.ones((h, w), device=device, dtype=torch.float32) # 假设生成一个[3, 3]的全为1的矩阵，命名为grid,并存储在device设备上，以便后续运算更新
        x_embed = grid.cumsum(dim=1) - 0.5 # x轴坐标embedding为grid按行累加并减0.5，即每一行为[0.5, 1.5, 2.5]，shape = [3, 3]
        y_embed = grid.cumsum(dim=0) - 0.5 # y轴坐标embedding为grid按列累加并减0.5，即每一列为[0.5, 1.5, 2.5]，shape = [3, 3]
        x_embed = x_embed / w # 标准化，使其规范为[0, 1]之间，shape = (3, 3)
        y_embed = y_embed / h # 标准化，使其规范为[0, 1]之间，shape = (3, 3)
        
        coords = torch.stack([x_embed, y_embed], dim=-1) # 在最后一列创建一个新的维度，stack在一起，得到coords，shape为[3, 3, 2]
        position_embedding = self._position_embedding_encoding(coords) # 再将coords传入_position_embedding_encoding，得到position_embedding，shape = [3, 3, 128]
        return position_embedding.permute(2, 0, 1)  # 调整为[128, 3, 3]，也就是 [C, H, W]

    # 根据实际输入的坐标的shape和原始图像shape，标准化之后进行随机位置编码
    # 传入的是坐标，即coords，和原始图像大小，即image_size
    def forward_with_coords(self, 
                            coords_input: torch.Tensor, # shape = (batch_size, N, 2)
                            image_size: Tuple[int, int]
                           ) -> torch.Tensor:

        coords = coords_input.clone()
        coords[:, :, 0] = coords[:, :, 0] / image_size[1] # 标准化x轴的坐标
        coords[:, :, 1] = coords[:, :, 1] / image_size[0] # 标准化y轴的坐标

        position_embedding = self._position_embedding_encoding(coords.to(torch.float)) # B x N x C
        
        return position_embedding.permute(2, 0, 1) # 原版代码忘了permute了

In [36]:
# 测试
positionembeddinger = PositionEmbeddingRandom()

# 测试1
coords_size = (3, 4) # batch_size = 3, N = 4
output1 = positionembeddinger(coords_size)
# 注意，由于PositionEmbeddingRandom()继承了nn.Module
# 因此这里positionembeddinger(coords)自动调用了该类的forward函数
# 即等价于positionembeddinger.forward(coords)

print(output1.shape)
print(output1)

# 测试2，shape = [2, 3, 2]
coords_input = torch.tensor([[[50, 50], [100, 100], [150, 150]],  # 第一个样本的坐标
                             [[30, 40], [70, 80], [110, 120]]])   # 第二个样本的坐标

# 定义图像的原始大小
image_size = (200, 300)  # 例如图像的大小为 200x300 (height, width)

# 调用 forward_with_coords 方法
output2 = positionembeddinger.forward_with_coords(coords_input, image_size)
print(output2.shape)
print(output2)

torch.Size([128, 3, 4])
tensor([[[ 0.8295, -0.1020, -0.6989,  0.9969],
         [ 0.2539, -0.9056,  0.9056, -0.2539],
         [-0.9969,  0.6989,  0.1020, -0.8295]],

        [[-0.0094,  0.2648,  0.5188,  0.7331],
         [-0.4043, -0.1383,  0.1383,  0.4043],
         [-0.7331, -0.5188, -0.2648,  0.0094]],

        [[ 0.9984, -0.6216, -0.2817,  0.9464],
         [ 0.1359, -0.8878,  0.8878, -0.1359],
         [-0.9464,  0.2817,  0.6216, -0.9984]],

        ...,

        [[-0.1795, -0.9445,  0.4699,  0.7999],
         [-0.8505,  0.6505,  0.6505, -0.8505],
         [ 0.7999,  0.4699, -0.9445, -0.1795]],

        [[ 0.7271,  0.0135, -0.7084, -0.9999],
         [ 0.3614,  0.9209,  0.9209,  0.3614],
         [-0.9999, -0.7084,  0.0135,  0.7271]],

        [[-0.5947,  0.6756, -0.7490,  0.8143],
         [-0.1567,  0.0524,  0.0524, -0.1567],
         [ 0.8143, -0.7490,  0.6756, -0.5947]]])
torch.Size([128, 2, 3])
tensor([[[-0.8948, -0.8948, -0.8948],
         [-0.8948, -0.8948, -0.8948]],

  